In [1]:
!pip install nbformat>=4.2.0

In [2]:

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime, timedelta
import warnings
from typing import Dict, Tuple, Optional
from scipy import stats
import yfinance as yf


# Machine Learning Imports
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

# Try importing advanced libraries
try:
    import xgboost as xgb
    HAS_XGB = True
except ImportError:
    HAS_XGB = False
    print("XGBoost not available - using baseline models only")

try:
    import shap
    HAS_SHAP = True
except ImportError:
    HAS_SHAP = False
    print("SHAP not available - feature importance will be limited")

warnings.filterwarnings('ignore')

a:\PROJECTS\MTF_TRADING_SYSTEM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
"""
Production-Grade Forecasting Agent - Fixed Version
Enhanced 30-day stock price forecasting with proper visualization
"""



class ForecastingAgent:
    """Production-grade forecasting with proper visualization."""
    
    def __init__(self, symbol: str, forecast_days: int = 30):
        self.symbol = symbol
        self.forecast_days = forecast_days
        self.models = {}
        self.scalers = {}
        self.feature_cols = []
        self.historical_volatility = 0
        self.historical_return = 0
        
    def fetch_data(self, period: str = "3y") -> pd.DataFrame:
        """Fetch historical data using yfinance directly."""
        try:
            # Fetch data
            ticker = yf.Ticker(self.symbol)
            df = ticker.history(period=period)
            
            if df.empty or len(df) < 100:
                raise ValueError(f"Insufficient data for {self.symbol}")
            
            # Reset index to get Date as column
            df = df.reset_index()
            
            # Ensure required columns exist
            required_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
            for col in required_cols:
                if col not in df.columns:
                    raise ValueError(f"Missing column: {col}")
            
            # Keep only required columns
            df = df[required_cols].copy()
            
            # Convert to numeric
            for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            
            # Drop NaN
            df = df.dropna()
            
            # Remove outliers (>5 sigma moves)
            if len(df) > 20:
                returns = df['Close'].pct_change()
                valid_returns = returns.fillna(0)
                if valid_returns.std() > 0:
                    z_scores = np.abs(stats.zscore(valid_returns))
                    df = df[z_scores < 5].copy()
            
            return df
            
        except Exception as e:
            raise ValueError(f"Failed to fetch data: {str(e)}")
    
    def calculate_technical_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Enhanced feature engineering."""
        df = df.copy()
        
        # Basic price features
        df['Returns'] = df['Close'].pct_change()
        df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
        
        # Lagged returns
        for lag in [1, 2, 3, 5]:
            df[f'Return_Lag{lag}'] = df['Returns'].shift(lag)
        
        # Multi-period returns
        for period in [5, 10, 20]:
            df[f'Return_{period}d'] = df['Close'].pct_change(period)
        
        # Volatility
        for window in [5, 10, 20]:
            df[f'Volatility_{window}d'] = df['Returns'].rolling(window).std()
        
        # Moving averages
        for window in [5, 10, 20, 50]:
            df[f'SMA_{window}'] = df['Close'].rolling(window).mean()
            df[f'EMA_{window}'] = df['Close'].ewm(span=window, adjust=False).mean()
            
            # Avoid division by zero
            sma_col = df[f'SMA_{window}'].replace(0, np.nan)
            ema_col = df[f'EMA_{window}'].replace(0, np.nan)
            
            df[f'Price_to_SMA_{window}'] = (df['Close'] - sma_col) / sma_col
            df[f'Price_to_EMA_{window}'] = (df['Close'] - ema_col) / ema_col
        
        # RSI
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss.replace(0, np.nan)
        df['RSI'] = 100 - (100 / (1 + rs))
        df['RSI'] = df['RSI'].fillna(50)
        
        # MACD
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        df['MACD'] = exp1 - exp2
        df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
        df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
        
        # Bollinger Bands
        window = 20
        df['BB_Middle'] = df['Close'].rolling(window).mean()
        df['BB_Std'] = df['Close'].rolling(window).std()
        df['BB_Upper'] = df['BB_Middle'] + 2 * df['BB_Std']
        df['BB_Lower'] = df['BB_Middle'] - 2 * df['BB_Std']
        
        # Avoid division by zero
        bb_middle = df['BB_Middle'].replace(0, np.nan)
        bb_range = (df['BB_Upper'] - df['BB_Lower']).replace(0, np.nan)
        
        df['BB_Width'] = (df['BB_Upper'] - df['BB_Lower']) / bb_middle
        df['BB_Position'] = (df['Close'] - df['BB_Lower']) / bb_range
        
        # Volume indicators
        if 'Volume' in df.columns:
            df['Volume_SMA_20'] = df['Volume'].rolling(20).mean()
            vol_sma = df['Volume_SMA_20'].replace(0, np.nan)
            df['Volume_Ratio'] = df['Volume'] / vol_sma
        
        # Price momentum
        df['ROC_10'] = df['Close'].pct_change(10) * 100
        df['ROC_20'] = df['Close'].pct_change(20) * 100
        
        # High-Low ratio
        df['High_Low_Ratio'] = df['High'] / df['Low'].replace(0, np.nan)
        
        # Price range percentage
        df['Price_Range_Pct'] = (df['High'] - df['Low']) / df['Low'].replace(0, np.nan)
        
        # Clean infinities
        df = df.replace([np.inf, -np.inf], np.nan)
        
        # Fill NaN with 0 for most columns, with appropriate defaults for indicators
        for col in df.columns:
            if col not in ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']:
                if 'RSI' in col:
                    df[col] = df[col].fillna(50)
                elif 'BB_Position' in col:
                    df[col] = df[col].fillna(0.5)
                else:
                    df[col] = df[col].fillna(0)
        
        return df
    
    def prepare_training_data(self, df: pd.DataFrame) -> Tuple[np.ndarray, np.ndarray, pd.DataFrame]:
        """Prepare features for training."""
        
        # Add technical features
        df = self.calculate_technical_features(df)
        
        # Create target: future price (not return)
        # Predict the price 'forecast_days' ahead
        df['Target'] = df['Close'].shift(-self.forecast_days)
        
        # Drop NaN values
        df = df.dropna()
        
        if len(df) < 50:
            raise ValueError(f"Insufficient data after feature engineering: {len(df)} rows")
        
        # Select features (exclude price columns and target)
        exclude_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Target',
                       'SMA_5', 'SMA_10', 'SMA_20', 'SMA_50',
                       'EMA_5', 'EMA_10', 'EMA_20', 'EMA_50',
                       'BB_Upper', 'BB_Lower', 'BB_Middle', 'Volume_SMA_20']
        
        self.feature_cols = [col for col in df.columns if col not in exclude_cols]
        
        if len(self.feature_cols) == 0:
            raise ValueError("No features generated")
        
        # Historical statistics
        if len(df) > 1:
            returns = df['Close'].pct_change().dropna()
            if len(returns) > 0:
                self.historical_volatility = returns.std() * np.sqrt(252) if returns.std() > 0 else 0.2
                self.historical_return = returns.mean() * 252 if len(returns) > 0 else 0.05
        else:
            self.historical_volatility = 0.2
            self.historical_return = 0.05
        
        # Prepare features and target
        X = df[self.feature_cols].values
        y = df['Target'].values
        
        # Ensure all values are finite
        finite_mask = np.isfinite(X).all(axis=1) & np.isfinite(y)
        X = X[finite_mask]
        y = y[finite_mask]
        df = df.iloc[finite_mask]
        
        if len(X) == 0:
            raise ValueError("No valid data points after removing non-finite values")
        
        # Scale features
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        self.scalers['X'] = scaler
        
        return X_scaled, y, df
    
    def train_ensemble(self, X_train, y_train, X_val, y_val):
        """Train ensemble of models."""
        
        # 1. Ridge Regression
        ridge = Ridge(alpha=1.0, random_state=42)
        ridge.fit(X_train, y_train)
        self.models['ridge'] = ridge
        
        # 2. XGBoost (if available)
        if HAS_XGB and len(X_train) > 10:
            try:
                # FIXED: early_stopping_rounds goes in constructor, not fit()
                xgb_model = xgb.XGBRegressor(
                    n_estimators=200,
                    learning_rate=0.05,
                    max_depth=4,  # Reduced to prevent overfitting
                    min_child_weight=2,
                    subsample=0.8,
                    colsample_bytree=0.8,
                    objective='reg:squarederror',
                    random_state=42,
                    n_jobs=-1,
                    early_stopping_rounds=20
                )
                
                # Fit with validation set for early stopping
                xgb_model.fit(
                    X_train, 
                    y_train,
                    eval_set=[(X_val, y_val)],
                    verbose=False
                )
                self.models['xgb'] = xgb_model
            except Exception as e:
                print(f"XGBoost training warning: {e}")
                # Continue without XGBoost
    
    def predict_ensemble(self, X: np.ndarray) -> float:
        """Weighted ensemble prediction."""
        predictions = []
        weights = []
        
        # Ridge prediction
        if 'ridge' in self.models:
            try:
                pred = self.models['ridge'].predict(X.reshape(1, -1))[0]
                predictions.append(pred)
                weights.append(0.4)
            except:
                pass
        
        # XGBoost prediction
        if 'xgb' in self.models:
            try:
                pred = self.models['xgb'].predict(X.reshape(1, -1))[0]
                predictions.append(pred)
                weights.append(0.6)
            except:
                pass
        
        if not predictions:
            # Fallback: use last known price
            return 0.0
            
        # Weighted average
        ensemble_pred = np.average(predictions, weights=weights)
        
        return float(ensemble_pred)
    
    def evaluate_models(self, X_val, y_val) -> Dict:
        """Model evaluation."""
        predictions = []
        
        # Get predictions from each model
        if 'ridge' in self.models:
            try:
                predictions.append(self.models['ridge'].predict(X_val))
            except:
                pass
        
        if 'xgb' in self.models:
            try:
                predictions.append(self.models['xgb'].predict(X_val))
            except:
                pass
        
        if not predictions or len(predictions[0]) == 0:
            return {
                'rmse': 0.0,
                'mae': 0.0,
                'r2': 0.0,
                'direction_accuracy': 0.5
            }
            
        # Ensemble prediction
        y_pred = np.mean(predictions, axis=0)
        
        # Metrics
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        
        # Directional accuracy
        if len(y_val) > 1 and len(y_pred) > 1:
            min_len = min(len(y_val), len(y_pred))
            direction_true = np.sign(np.diff(y_val[:min_len]))
            direction_pred = np.sign(np.diff(y_pred[:min_len]))
            dir_accuracy = np.mean(direction_true == direction_pred)
        else:
            dir_accuracy = 0.5
        
        return {
            'rmse': float(rmse),
            'mae': float(mae),
            'r2': float(r2),
            'direction_accuracy': float(dir_accuracy)
        }
    
    def generate_forecast(self, df: pd.DataFrame, predicted_price: float) -> Dict:
        """Generate forecast with confidence intervals."""
        
        current_price = float(df['Close'].iloc[-1])
        predicted_return = (predicted_price / current_price) - 1
        
        # Generate dates for forecast - FIXED VERSION
        last_date = df['Date'].iloc[-1]
        forecast_dates = []
        current_date = pd.Timestamp(last_date)  # Ensure it's a pandas Timestamp

        for i in range(self.forecast_days + 1):
            # Skip weekends (stock market is closed)
            while current_date.weekday() >= 5:
                current_date = current_date + pd.Timedelta(days=1)
            forecast_dates.append(current_date)
            current_date = current_date + pd.Timedelta(days=1)
        
        # Generate smooth path from current to predicted price
        days = np.arange(self.forecast_days + 1)
        
        # Create a smooth path
        # Using cubic interpolation for natural-looking curve
        t = days / self.forecast_days
        
        # Base path (linear interpolation)
        base_path = current_price + (predicted_price - current_price) * t
        
        # Add some gentle noise that decreases towards the end
        noise_scale = 0.02 * current_price * (1 - t)
        noise = np.random.normal(0, noise_scale)
        
        # Smooth the noise
        if len(noise) > 3:
            noise = pd.Series(noise).rolling(3, center=True, min_periods=1).mean().values
        
        price_path = base_path + noise
        
        # Ensure the path ends at predicted price
        price_path[-1] = predicted_price
        
        # Confidence intervals (95%)
        # Use historical volatility scaled by time
        daily_vol = self.historical_volatility / np.sqrt(252)
        cumulative_vol = daily_vol * np.sqrt(days + 1)
        
        upper_bound = price_path * (1 + 1.96 * cumulative_vol)
        lower_bound = price_path * (1 - 1.96 * cumulative_vol)
        
        return {
            'dates': forecast_dates,
            'mean_path': price_path.tolist(),
            'upper_band': upper_bound.tolist(),
            'lower_band': lower_bound.tolist(),
            'final_price': float(predicted_price),
            'predicted_return_pct': float(predicted_return * 100),
            'current_price': float(current_price)
        }
    
    def run_pipeline(self) -> Dict:
        """Execute complete forecasting pipeline."""
        
        try:
            # 1. Fetch data
            raw_df = self.fetch_data(period="2y")  # Use 2 years for faster training
            
            if len(raw_df) < 60:
                raise ValueError(f"Insufficient historical data: {len(raw_df)} rows")
            
            # 2. Prepare features
            X, y, df = self.prepare_training_data(raw_df)
            
            # 3. Train/Val split (use time-based split)
            split_idx = int(len(X) * 0.8)
            X_train, X_val = X[:split_idx], X[split_idx:]
            y_train, y_val = y[:split_idx], y[split_idx:]
            
            if len(X_train) < 10 or len(X_val) < 5:
                raise ValueError("Not enough data for training/validation split")
            
            # 4. Train models
            self.train_ensemble(X_train, y_train, X_val, y_val)
            
            # 5. Evaluate
            metrics = self.evaluate_models(X_val, y_val)
            
            # 6. Generate forecast using latest data
            if len(df) == 0:
                raise ValueError("No data available for prediction")
            
            latest_features = df[self.feature_cols].iloc[[-1]].values
            if latest_features.shape[1] != X_train.shape[1]:
                raise ValueError(f"Feature dimension mismatch: {latest_features.shape[1]} vs {X_train.shape[1]}")
            
            X_latest = self.scalers['X'].transform(latest_features)
            
            predicted_price = self.predict_ensemble(X_latest[0])
            
            # Sanity check on prediction
            current_price = float(df['Close'].iloc[-1])
            max_change = 0.5  # Max 50% change
            if abs(predicted_price - current_price) / current_price > max_change:
                # Clip to reasonable range
                if predicted_price > current_price:
                    predicted_price = current_price * (1 + max_change)
                else:
                    predicted_price = current_price * (1 - max_change)
            
            # 7. Generate forecast path
            forecast = self.generate_forecast(df, predicted_price)
            
            # 8. Feature Importance (if XGBoost and SHAP available)
            shap_values = None
            if 'xgb' in self.models and HAS_SHAP:
                try:
                    explainer = shap.TreeExplainer(self.models['xgb'])
                    shap_values = explainer.shap_values(X_latest)
                    # Flatten if needed
                    if hasattr(shap_values, 'flatten'):
                        shap_values = shap_values.flatten()
                except Exception as e:
                    print(f"SHAP calculation warning: {e}")
                    shap_values = None
            
            # 9. Return results
            return {
                'success': True,
                'metrics': metrics,
                'forecast': forecast,
                'history': df[['Date', 'Close']].tail(60).reset_index(drop=True),
                'models_used': list(self.models.keys()),
                'feature_count': len(self.feature_cols),
                'shap_values': shap_values,
                'feature_names': self.feature_cols,
                'current_price': current_price
            }
            
        except Exception as e:
            import traceback
            print(f"Forecast pipeline error: {e}")
            return {
                'success': False,
                'error': str(e)
            }
    
    def create_forecast_chart(self, result):
        """Create an interactive forecast visualization"""
        # Get forecast data
        forecast = result['forecast']
        dates = forecast['dates']
        predicted = forecast['mean_path']
        upper_band = forecast['upper_band']
        lower_band = forecast['lower_band']
        current_price = forecast['current_price']
        final_price = forecast['final_price']
        predicted_return = forecast['predicted_return_pct']
        
        fig = go.Figure()
        
        # Predicted future prices
        fig.add_trace(go.Scatter(
            x=dates,
            y=predicted,
            mode='lines+markers',
            name='Predicted Price Path',
            line=dict(color='#ff6b6b', width=3),
            marker=dict(size=6, color='#ff6b6b'),
            hovertemplate='<b>Predicted</b><br>Date: %{x}<br>Price: ₹%{y:.2f}<extra></extra>'
        ))
        
        # Confidence Band
        fig.add_trace(go.Scatter(
            x=list(dates) + list(dates)[::-1],
            y=list(upper_band) + list(lower_band)[::-1],
            fill='toself',
            fillcolor='rgba(255, 107, 107, 0.2)',
            line=dict(color='rgba(255,255,255,0)'),
            showlegend=True,
            name='Confidence Band (±2%)',
            hoverinfo='skip'
        ))
        
        # Add current price as starting point
        fig.add_trace(go.Scatter(
            x=[dates[0]],
            y=[current_price],
            mode='markers',
            name='Current Price',
            marker=dict(size=15, color='#00d4ff', symbol='star'),
            hovertemplate='<b>Current Price</b><br>₹%{y:.2f}<extra></extra>'
        ))
        
        # Add target price as ending point
        fig.add_trace(go.Scatter(
            x=[dates[-1]],
            y=[final_price],
            mode='markers',
            name='Target Price',
            marker=dict(size=15, color='#00ff00', symbol='star'),
            hovertemplate='<b>Target Price</b><br>₹%{y:.2f}<extra></extra>'
        ))
        
        # Add annotations
        fig.add_annotation(
            x=dates[0],
            y=current_price,
            text=f"Current: ₹{current_price:.2f}",
            showarrow=True,
            arrowhead=2,
            arrowcolor='#00d4ff',
            ax=-60,
            ay=-40,
            font=dict(color='white', size=12, family='Arial Black'),
            bgcolor='rgba(0, 212, 255, 0.3)',
            bordercolor='#00d4ff',
            borderwidth=2
        )
        
        fig.add_annotation(
            x=dates[-1],
            y=final_price,
            text=f"Target: ₹{final_price:.2f}<br>({predicted_return:+.2f}%)",
            showarrow=True,
            arrowhead=2,
            arrowcolor='#00ff00',
            ax=60,
            ay=-40,
            font=dict(color='white', size=12, family='Arial Black'),
            bgcolor='rgba(0, 255, 0, 0.3)',
            bordercolor='#00ff00',
            borderwidth=2
        )
        
        # Color code title based on prediction
        title_color = '#00ff00' if predicted_return > 0 else '#ff6b6b'
        
        fig.update_layout(
            title={
                'text': f'📈 Stock Price Forecast | Return: <span style="color:{title_color}">{predicted_return:+.2f}%</span>',
                'x': 0.5,
                'xanchor': 'center',
                'font': {'size': 24, 'color': 'white', 'family': 'Arial Black'}
            },
            xaxis_title='Date',
            yaxis_title='Price (₹)',
            template='plotly_dark',
            hovermode='x unified',
            plot_bgcolor='#1a1a2e',
            paper_bgcolor='#16213e',
            font=dict(color='white', size=12),
            height=600,
            showlegend=True,
            legend=dict(
                x=0.01,
                y=0.99,
                bgcolor='rgba(0,0,0,0.5)',
                bordercolor='white',
                borderwidth=1
            ),
            xaxis=dict(
                showgrid=True,
                gridcolor='rgba(255,255,255,0.1)'
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor='rgba(255,255,255,0.1)'
            )
        )
        
        return fig


    def create_feature_importance_chart(self, result: Dict) -> go.Figure:
        """Create feature importance visualization."""
        
        # Fixed: Check for None or missing key explicitly
        if not result.get('success', False) or 'shap_values' not in result or result['shap_values'] is None:
            # Return empty figure with message
            fig = go.Figure()
            fig.add_annotation(
                text="Feature importance not available",
                xref="paper", yref="paper",
                x=0.5, y=0.5, showarrow=False,
                font=dict(size=14, color='#9ca3af')
            )
            fig.update_layout(
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                height=400
            )
            return fig
        
        shap_values = result['shap_values']
        feature_names = result['feature_names']
        
        # Convert to numpy array if needed
        if not isinstance(shap_values, np.ndarray):
            shap_values = np.array(shap_values)
        
        # Flatten if multi-dimensional
        if shap_values.ndim > 1:
            shap_values = shap_values.flatten()
        
        # Get absolute values for importance
        abs_shap = np.abs(shap_values)
        
        # Get top features
        top_n = min(10, len(feature_names))
        top_indices = np.argsort(abs_shap)[-top_n:][::-1]
        top_features = [feature_names[i] for i in top_indices]
        top_values = shap_values[top_indices]
        
        # Colors (green for positive impact, red for negative)
        colors = ['#22c55e' if v > 0 else '#ef4444' for v in top_values]
        
        fig = go.Figure()
        
        fig.add_trace(go.Bar(
            y=top_features,
            x=top_values,
            orientation='h',
            marker=dict(color=colors),
            hovertemplate='<b>%{y}</b><br>Impact: %{x:.4f}<extra></extra>'
        ))
        
        fig.update_layout(
            title=dict(
                text='🎯 Top Features Driving Prediction',
                font=dict(size=16, color='#e5e7eb'),
                x=0.5
            ),
            xaxis=dict(
                title='SHAP Value (Impact on Prediction)',
                title_font=dict(size=12, color='#e5e7eb'),  # Fixed: title_font instead of titlefont
                tickfont=dict(size=10, color='#e5e7eb'),
                gridcolor='rgba(255,255,255,0.1)',
                showgrid=True,
                zeroline=True,
                zerolinecolor='rgba(255,255,255,0.3)',
                zerolinewidth=2
            ),
            yaxis=dict(
                title_font=dict(size=12, color='#e5e7eb'),  # Fixed: title_font instead of titlefont
                tickfont=dict(size=10, color='#e5e7eb'),
                showgrid=False
            ),
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font=dict(color='#e5e7eb'),
            height=400,
            margin=dict(l=200, r=40, t=60, b=40),
            showlegend=False
        )
        
        return fig


In [4]:
agent = ForecastingAgent("RELIANCE.NS", forecast_days=30)
result = agent.run_pipeline()

In [5]:
result.keys()

dict_keys(['success', 'metrics', 'forecast', 'history', 'models_used', 'feature_count', 'shap_values', 'feature_names', 'current_price'])

In [6]:
result["forecast"].keys()

dict_keys(['dates', 'mean_path', 'upper_band', 'lower_band', 'final_price', 'predicted_return_pct', 'current_price'])

In [7]:


# Test the agent
if __name__ == "__main__":
    # Test with a popular Indian stock
    agent = ForecastingAgent("RELIANCE.NS", forecast_days=30)
    result = agent.run_pipeline()
    
    if result.get('success', False):
        print(f"Forecast successful!")
        print(f"Current Price: ₹{result['forecast']['current_price']:.2f}")
        print(f"Predicted Price: ₹{result['forecast']['final_price']:.2f}")
        print(f"Predicted Return: {result['forecast']['predicted_return_pct']:.2f}%")
        print(f"Models Used: {result['models_used']}")
        print(f"RMSE: {result['metrics']['rmse']:.4f}")
        print(f"Direction Accuracy: {result['metrics']['direction_accuracy']*100:.1f}%")
        
        # Create and show charts
        forecast_fig = agent.create_forecast_chart(result)
        forecast_fig.show()
        
        importance_fig = agent.create_feature_importance_chart(result)
        importance_fig.show()
    else:
        print(f"Forecast failed: {result.get('error', 'Unknown error')}")

Forecast successful!
Current Price: ₹1484.10
Predicted Price: ₹1400.63
Predicted Return: -5.62%
Models Used: ['ridge', 'xgb']
RMSE: 97.1934
Direction Accuracy: 56.5%
